In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score

# Tensorflow dependencies
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import regularizers
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.models import Model

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

data = pd.read_csv('/content/drive/My Drive/secondary_data.csv',sep=';')

Mounted at /content/drive


In [ ]:
print(data)

      class  cap-diameter cap-shape cap-surface cap-color  \
0         p         15.26         x           g         o   
1         p         16.60         x           g         o   
2         p         14.07         x           g         o   
3         p         14.17         f           h         e   
4         p         14.64         x           h         o   
...     ...           ...       ...         ...       ...   
61064     p          1.18         s           s         y   
61065     p          1.27         f           s         y   
61066     p          1.27         s           s         y   
61067     p          1.24         f           s         y   
61068     p          1.17         s           s         y   

      does-bruise-or-bleed gill-attachment gill-spacing gill-color  \
0                        f               e          NaN          w   
1                        f               e          NaN          w   
2                        f               e          NaN  

In [ ]:
data = data = pd.get_dummies(data)

In [ ]:
X = data.drop(columns=['class_p', 'class_e']).values
y = data['class_p'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [ ]:
#ANN Architecture
class ANNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, in_features=10, out_features=2, num_hidden=1,
                 num_neurons=16, regularizer='l2',
                 reg_param=1e-4, batch_norm=False, epochs=100,
                 early_stopping=True, verbose=1):
        self.in_features = in_features
        self.out_features = out_features
        self.num_hidden = num_hidden
        self.num_neurons = num_neurons
        self.regularizer = regularizer
        self.regularizer_func = regularizers.l2 if regularizer=='l2' else regularizers.l1
        self.reg_param = reg_param
        self.batch_norm = batch_norm
        self.epochs = epochs
        self.early_stopping = early_stopping
        self.verbose = verbose

        # Build the ANN
        self.model = ANNClassifier.build_model(in_features, out_features,
                                               self.num_hidden,
                                               self.num_neurons,
                                               self.regularizer_func,
                                               self.reg_param,
                                               self.batch_norm)

    @staticmethod
    def build_model(in_features, out_features,
                    num_hidden, num_neurons,
                    regularizer_func, reg_param, batch_norm):
        inputs = Input(shape=(in_features, ))
        outputs = inputs

        for i in range(num_hidden):
            outputs = Dense(num_neurons, kernel_regularizer=regularizer_func(reg_param), activation='relu')(outputs)
            if(batch_norm):
                outputs = BatchNormalization()(outputs)

        outputs = Dense(out_features, activation='softmax')(outputs)
        model = Model(inputs=inputs, outputs=outputs, name='ANNClassifier')

        return model

    def fit(self, X, Y):
        # Split into training and validating sets
        X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=1/3)

        # Specifies callbacks list
        callbacks = [
            ModelCheckpoint('model.weights.hdf5', save_best_only=True, verbose=self.verbose)
        ]

        # If early stopping is on, append early stopping callback to list
        if(self.early_stopping):
            callbacks.append(EarlyStopping(monitor='val_loss', patience=10, verbose=self.verbose))

        # Configure training process
        adam = Adam(learning_rate=0.001, amsgrad=True)
        loss = SparseCategoricalCrossentropy()
        accuracy = SparseCategoricalAccuracy()

        # Compile the model then train
        self.model.compile(optimizer=adam, loss=loss, metrics=[accuracy])
        history = self.model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=self.epochs,
                       callbacks=callbacks, verbose=self.verbose)

        train_loss = history.history['loss'][-1]
        val_loss = history.history['val_loss'][-1]
        print("Training Loss:", train_loss)
        print("Validation Loss:", val_loss)

        # Calculate training and validation accuracy
        train_accuracy = history.history['sparse_categorical_accuracy'][-1]
        val_accuracy = history.history['val_sparse_categorical_accuracy'][-1]

        # Print or return the metrics
        print("Training Accuracy:", train_accuracy)
        print("Validation Accuracy:", val_accuracy)

        return history

    def predict(self, X):
        predictions = self.model.predict(X)
        predictions = np.argmax(predictions, axis=1)

        return predictions


In [ ]:
exclude = ['class_p','class_e']
features = data.drop(columns=exclude)
in_features = features.shape[1]
out_features = 2

In [ ]:
ann = ANNClassifier(in_features, out_features, epochs=10, verbose=1)
ann.fit(X_train, y_train)

Epoch 1/10
834/853 [============================>.] - ETA: 0s - loss: 0.4253 - sparse_categorical_accuracy: 0.8249
Epoch 1: val_loss improved from inf to 0.21764, saving model to model.weights.hdf5
853/853 [==============================] - 4s 3ms/step - loss: 0.4211 - sparse_categorical_accuracy: 0.8273 - val_loss: 0.2176 - val_sparse_categorical_accuracy: 0.9426
Epoch 2/10
852/853 [============================>.] - ETA: 0s - loss: 0.1489 - sparse_categorical_accuracy: 0.9712
Epoch 2: val_loss improved from 0.21764 to 0.09922, saving model to model.weights.hdf5
853/853 [==============================] - 2s 3ms/step - loss: 0.1488 - sparse_categorical_accuracy: 0.9713 - val_loss: 0.0992 - val_sparse_categorical_accuracy: 0.9868
Epoch 3/10
831/853 [============================>.] - ETA: 0s - loss: 0.0780 - sparse_categorical_accuracy: 0.9914
Epoch 3: val_loss improved from 0.09922 to 0.05936, saving model to model.weights.hdf5
853/853 [==============================] - 2s 3ms/step - los

In [ ]:
param_grid = {
    'num_hidden' : [1, 2],
    'num_neurons' : [8, 16],
    'reg_param' : [2e-4, 2e-3]
}

clf = ANNClassifier(in_features=in_features, out_features=out_features, verbose=1, epochs=3)
gridsearchcv = GridSearchCV(clf, param_grid, cv=3, scoring='accuracy', return_train_score=True, verbose=3)
gridsearchcv.fit(X_train, y_train)
best_params = gridsearchcv.best_params_

Fitting 3 folds for each of 8 candidates, totalling 24 fits
Epoch 1/3
562/569 [============================>.] - ETA: 0s - loss: 0.5056 - sparse_categorical_accuracy: 0.7504
Epoch 1: val_loss improved from inf to 0.34283, saving model to model.weights.hdf5
569/569 [==============================] - 3s 5ms/step - loss: 0.5043 - sparse_categorical_accuracy: 0.7512 - val_loss: 0.3428 - val_sparse_categorical_accuracy: 0.8635
Epoch 2/3
 49/569 [=>............................] - ETA: 1s - loss: 0.3219 - sparse_categorical_accuracy: 0.8903

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


551/569 [============================>.] - ETA: 0s - loss: 0.2635 - sparse_categorical_accuracy: 0.9115
Epoch 2: val_loss improved from 0.34283 to 0.19670, saving model to model.weights.hdf5
569/569 [==============================] - 2s 4ms/step - loss: 0.2614 - sparse_categorical_accuracy: 0.9126 - val_loss: 0.1967 - val_sparse_categorical_accuracy: 0.9495
Epoch 3/3
565/569 [============================>.] - ETA: 0s - loss: 0.1614 - sparse_categorical_accuracy: 0.9604
Epoch 3: val_loss improved from 0.19670 to 0.13383, saving model to model.weights.hdf5
569/569 [==============================] - 2s 3ms/step - loss: 0.1611 - sparse_categorical_accuracy: 0.9606 - val_loss: 0.1338 - val_sparse_categorical_accuracy: 0.9718
Training Loss: 0.16108502447605133
Validation Loss: 0.13382774591445923
Training Accuracy: 0.9605697393417358
Validation Accuracy: 0.9718464612960815
853/853 [==============================] - 1s 2ms/step
[CV 1/3] END num_hidden=1, num_neurons=8, reg_param=0.0002;, scor

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


552/569 [============================>.] - ETA: 0s - loss: 0.2630 - sparse_categorical_accuracy: 0.9168
Epoch 2: val_loss improved from 0.34193 to 0.20066, saving model to model.weights.hdf5
569/569 [==============================] - 2s 4ms/step - loss: 0.2617 - sparse_categorical_accuracy: 0.9168 - val_loss: 0.2007 - val_sparse_categorical_accuracy: 0.9426
Epoch 3/3
562/569 [============================>.] - ETA: 0s - loss: 0.1581 - sparse_categorical_accuracy: 0.9610
Epoch 3: val_loss improved from 0.20066 to 0.12498, saving model to model.weights.hdf5
569/569 [==============================] - 2s 3ms/step - loss: 0.1577 - sparse_categorical_accuracy: 0.9613 - val_loss: 0.1250 - val_sparse_categorical_accuracy: 0.9781
Training Loss: 0.15769754350185394
Validation Loss: 0.12497959285974503
Training Accuracy: 0.9612846374511719
Validation Accuracy: 0.9781150221824646
853/853 [==============================] - 1s 1ms/step
[CV 2/3] END num_hidden=1, num_neurons=8, reg_param=0.0002;, scor

...

565/569 [============================>.] - ETA: 0s - loss: 0.2455 - sparse_categorical_accuracy: 0.9199
Epoch 2: val_loss improved from 0.32797 to 0.18457, saving model to model.weights.hdf5
569/569 [==============================] - 2s 3ms/step - loss: 0.2452 - sparse_categorical_accuracy: 0.9199 - val_loss: 0.1846 - val_sparse_categorical_accuracy: 0.9424
Epoch 3/3
557/569 [============================>.] - ETA: 0s - loss: 0.1461 - sparse_categorical_accuracy: 0.9652
Epoch 3: val_loss improved from 0.18457 to 0.11908, saving model to model.weights.hdf5
569/569 [==============================] - 2s 3ms/step - loss: 0.1456 - sparse_categorical_accuracy: 0.9653 - val_loss: 0.1191 - val_sparse_categorical_accuracy: 0.9781
Training Loss: 0.14555658400058746
Validation Loss: 0.11908017098903656
Training Accuracy: 0.9652991890907288
Validation Accuracy: 0.9781150221824646
853/853 [==============================] - 1s 1ms/step
[CV 2/3] END num_hidden=2, num_neurons=16, reg_param=0.002;, scor

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


561/569 [============================>.] - ETA: 0s - loss: 0.2483 - sparse_categorical_accuracy: 0.9206
Epoch 2: val_loss improved from 0.32907 to 0.18622, saving model to model.weights.hdf5
569/569 [==============================] - 2s 3ms/step - loss: 0.2476 - sparse_categorical_accuracy: 0.9208 - val_loss: 0.1862 - val_sparse_categorical_accuracy: 0.9473
Epoch 3/3
564/569 [============================>.] - ETA: 0s - loss: 0.1431 - sparse_categorical_accuracy: 0.9638
Epoch 3: val_loss improved from 0.18622 to 0.11481, saving model to model.weights.hdf5
569/569 [==============================] - 3s 4ms/step - loss: 0.1428 - sparse_categorical_accuracy: 0.9639 - val_loss: 0.1148 - val_sparse_categorical_accuracy: 0.9726
Training Loss: 0.1427573263645172
Validation Loss: 0.11481349915266037
Training Accuracy: 0.9639263153076172
Validation Accuracy: 0.9726163148880005
853/853 [==============================] - 1s 1ms/step
[CV 3/3] END num_hidden=2, num_neurons=16, reg_param=0.002;, score

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


847/853 [============================>.] - ETA: 0s - loss: 0.1551 - sparse_categorical_accuracy: 0.9891
Epoch 2: val_loss improved from 0.21065 to 0.12298, saving model to model.weights.hdf5
853/853 [==============================] - 3s 4ms/step - loss: 0.1549 - sparse_categorical_accuracy: 0.9891 - val_loss: 0.1230 - val_sparse_categorical_accuracy: 0.9970
Epoch 3/3
847/853 [============================>.] - ETA: 0s - loss: 0.1119 - sparse_categorical_accuracy: 0.9973
Epoch 3: val_loss improved from 0.12298 to 0.10213, saving model to model.weights.hdf5
853/853 [==============================] - 3s 4ms/step - loss: 0.1118 - sparse_categorical_accuracy: 0.9973 - val_loss: 0.1021 - val_sparse_categorical_accuracy: 0.9984
Training Loss: 0.11184718459844589
Validation Loss: 0.10212936252355576
Training Accuracy: 0.997323751449585
Validation Accuracy: 0.9983869791030884


In [ ]:
print('Best model parameters : ', best_params)

Best model parameters :  {'num_hidden': 2, 'num_neurons': 16, 'reg_param': 0.002}


In [ ]:
clf =  ANNClassifier(in_features=in_features, out_features=out_features, verbose=1, epochs=5, **best_params)
clf.fit(X_train, y_train)

Epoch 1/5
834/853 [============================>.] - ETA: 0s - loss: 0.4301 - sparse_categorical_accuracy: 0.8398
Epoch 1: val_loss improved from inf to 0.23085, saving model to model.weights.hdf5
853/853 [==============================] - 3s 3ms/step - loss: 0.4261 - sparse_categorical_accuracy: 0.8420 - val_loss: 0.2308 - val_sparse_categorical_accuracy: 0.9559
Epoch 2/5
849/853 [============================>.] - ETA: 0s - loss: 0.1760 - sparse_categorical_accuracy: 0.9771
Epoch 2: val_loss improved from 0.23085 to 0.14003, saving model to model.weights.hdf5
853/853 [==============================] - 3s 3ms/step - loss: 0.1759 - sparse_categorical_accuracy: 0.9771 - val_loss: 0.1400 - val_sparse_categorical_accuracy: 0.9911
Epoch 3/5
844/853 [============================>.] - ETA: 0s - loss: 0.1265 - sparse_categorical_accuracy: 0.9927
Epoch 3: val_loss improved from 0.14003 to 0.11392, saving model to model.weights.hdf5
853/853 [==============================] - 2s 3ms/step - loss: 

In [ ]:
predictions = clf.predict(X_test)
accuracy = accuracy_score(y_test, predictions)

print('Accuracy on testing set : ', accuracy)

630/630 [==============================] - 1s 2ms/step
Accuracy on testing set :  0.9974693594005856


Before Tuning

*   Training Loss: 0.022285955026745796
*   Validation Loss: 0.021776264533400536
*   Training Accuracy: 0.9998900294303894
*   Validation Accuracy: 0.9996334314346313

Best model parameters :  {'num_hidden': 2, 'num_neurons': 16, 'reg_param': 0.002}

After Tuning:

*   Training Loss: 0.09437078982591629
*   Validation Loss: 0.09008152782917023
*   Training Accuracy: 0.9971771240234375
*   Validation Accuracy: 0.9980937242507935



Accuracy on testing set :  0.9974693594005856